In [3]:
jobs=[2,3,4,5,6]
size=range(1,len(jobs)+1) 1-5
print(len(jobs))
for i in size:
    print(i)

5
1
2
3
4
5


In [ ]:
# 單筆
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os 
import plotly.express as px
import pandas as pd

# read file
filename= 'output_0-7'
machines_df = pd.read_csv("./"+ filename +"/machines.csv")
config_df = pd.read_csv("./"+ filename +"/config.csv")[['peak_period','minute_threshold','weight_setup_times','weight_cr']]
std_df =pd.read_csv(("./"+ filename +"/config.csv"))["std_time"]
result_df = pd.read_csv("./"+ filename +"/result.csv")

# time convert
input_scheduling_time= '2022-02-08 07:30' #開始算
std_time = int(time.mktime(time.strptime(std_df.iloc[0], "%y-%m-%d %H:%M")))
scheduling_time = int(time.mktime(time.strptime(input_scheduling_time, "%Y-%m-%d %H:%M")))
gap_time= round((scheduling_time - std_time)/60)
print(gap_time)
g_start_T= 0+ gap_time
g_end_T= 1440 + gap_time
rows=[]

# Get rid of repeative value
repeative_index = list(result_df[result_df.duplicated("lot_number",keep=False)].index)   #重複值包含空值!
delete_index =[]
for i in range(len(repeative_index)):
    if result_df.iloc[repeative_index[i]].at['oper']!=0: # keep lot number = 'NA'
        if result_df.iloc[repeative_index[i]].at['process_tsime'] !=0: # remove overlap job #刪掉非機台上有被排上的貨
            delete_index.append(repeative_index[i])
        
no_NA_df = result_df.drop(index=delete_index)
no_NA_df.reset_index(inplace=True, drop=True)

# Test range
#no_NA_df=no_NA_df[0:10]

#----------------------------
# 改機
def get_setup(no_NA_df):

    sorted_result_df=no_NA_df[no_NA_df["start_time"]<= g_end_T].sort_values(by=['entity', 'end_time']) #within one day
    setup_dict ={}
    times_info_list=[]
    entity_name= sorted_result_df['entity'].iloc[0] # default
    bd_id = sorted_result_df['bd_id'].iloc[0]       # default
    total_setup_times=0

    for i in range(len(sorted_result_df.index)): # serach for all data
        if entity_name == sorted_result_df['entity'].iloc[i]:  # If entity is same, need to judge whether bd_id is same. Else, record new entity(key)

            if bd_id != sorted_result_df['bd_id'].iloc[i]: # if bd_id is different need to record
                setup_start=sorted_result_df['end_time'].iloc[i-1]  #the endT of previous and convert to date
                setup_end=sorted_result_df['start_time'].iloc[i]  #the startT of current and convert to date
                # record
                if setup_end >= g_start_T: #**
                    times_info_list.append(str(setup_start)+' > '+str(setup_end)) 
                    total_setup_times +=1
                bd_id = sorted_result_df['bd_id'].iloc[i]

        else: 
            #add to dict
            setup_dict[entity_name]=times_info_list

            #next entity
            entity_name = sorted_result_df['entity'].iloc[i]
            bd_id = sorted_result_df['bd_id'].iloc[i]
            times_info_list=[]
    # test it
    #print(setup_dict)  
    return total_setup_times
    

# utilization
def get_utilization(no_NA_df):
    #oneday_result=no_NA_df[no_NA_df["start_time"]<= g_end_T]  # for 空機&稼動率
    oneday_result = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.start_time<=g_end_T)]
    entities=oneday_result.groupby("entity") #within one day
    entity_key_list= list(entities.groups.keys())
    total_processing_time=0
    total_processing_time_list=[]
    utilization_rate=0
    utilization_rate_dic={}
    utilization_rate_list=[]
    ## 一日空機
    count=0
    a = list(entities.size())
    for i in range(len(a)):
        if a[i]==1:
            count+=1

    #-----------------------------------------------------------------

    for i in range(len(entity_key_list)): #len(entity_key_list)
        key= entity_key_list[i]
        if i !=0: # first not in 
            total_processing_time_list.append(total_processing_time)
            #total_processing_time_list.append((entity_key_list[i-1],total_processing_time)) # test
            utilization_rate_dic[entity_key_list[i-1]] = round(total_processing_time/1440*100,2)
            total_processing_time=0

        for j in range(len(entities.groups[entity_key_list[i]])):
            row_index=entities.groups[key][j]
            if float(no_NA_df.iloc[row_index].at['start_time']) < g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) >g_end_T : # 
                processing_time = 1440
            elif float(no_NA_df.iloc[row_index].at['start_time']) < g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) <=g_end_T: # exceed one day
                processing_time = float(no_NA_df.iloc[row_index].at['end_time']) - g_start_T 
            elif float(no_NA_df.iloc[row_index].at['start_time']) >= g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) >g_end_T: # exceed one day
                processing_time = g_end_T - float(no_NA_df.iloc[row_index].at['start_time'])
            else: # normal 
                processing_time = float(no_NA_df.iloc[row_index].at['end_time']) - float(no_NA_df.iloc[row_index].at['start_time']) 

            total_processing_time += processing_time

        if i == len(entity_key_list)-1: # final one
            total_processing_time_list.append(total_processing_time)
            ## test it
            #total_processing_time_list.append((entity_key_list[i],total_processing_time)) # test
            utilization_rate_dic[entity_key_list[i]] = round(total_processing_time/1440*100,2)

    #print(len(total_processing_time_list))                                             # test
    #print(utilization_rate_dic)  

    # plot 
    u_df = pd.DataFrame(list(utilization_rate_dic.items()),columns=['Entity', 'Utilization_rate'])
    fig = px.bar(u_df, x='Entity', y='Utilization_rate', title='Distribution')
    fig.show()

    ratio= sum(total_processing_time_list)/(1440*len(machines_df.index))

    return "{:.2%}".format(ratio)
    

# 一日產量
def get_outplan(no_NA_df):
    outplan_df = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.end_time<=g_end_T)]
    outplan_df.reset_index(inplace=True, drop=True)
    outplan = 0
    for i in range(len(outplan_df.index)):
        outplan += outplan_df.iloc[i].at['qty']
    if outplan>1000:
        #print("產量:",round(outplan/1000),"K")
        return str(round(outplan/1000))+'K' 
    else:
        #print("產量:",outplan)
        return outplan 

# 排上機
def get_running_machines(no_NA_df,machines_df):
    outplan_df = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.start_time<=g_end_T)]
    outplan_df.reset_index(inplace=True, drop=True)    
    entities=outplan_df.groupby("entity") #within one day
    entity_key_list= list(entities.groups.keys())

    # total machine
    entities_2=machines_df.groupby("ENTITY") #within one day
    entity_key_list_2 = list(entities_2.groups.keys())

    # print no caculate utilization entity
    not_in_one_day = list(set(entity_key_list_2) - set(entity_key_list))
    print("Not running:",sorted(not_in_one_day))
    return "{}/{}".format(len(entity_key_list),len(entity_key_list_2)) 

rows.append([get_setup(no_NA_df),get_utilization(no_NA_df),get_outplan(no_NA_df),get_running_machines(no_NA_df,machines_df)])
  
indicator_df = pd.DataFrame(rows,columns=['Setup','Utilization','Outplan','Running_machines'])
final_df = pd.concat([config_df,indicator_df],axis=1)
# write to csv
#final_df.to_csv("indicator_"+ filename +".csv", index=False ,na_rep=0,encoding='utf_8_sig') 

final_df



In [5]:
i=1
print(str(i+2))

3


In [4]:
import plotly.express as px
import datetime
import pandas as pd

df=[]
  
df.append(dict(Task="J1", Start='2020-11-07 %s'%datetime.timedelta(minutes=10),
Finish='2020-11-07 %s'%datetime.timedelta(minutes=10000),Resource="M1"))


#呈現圖表
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Task",text="Task")
fig.show()


TypeError: Both x_start and x_end must refer to data convertible to datetimes.

In [3]:
import pandas as pd
import json
# from wip import WIP

class Job():
    def __init__(self, configure, processTime):
        self.configure = configure  #fix
        self.processTime = processTime[processTime["RECIPE"] == self.configure["RECIPE"]] #fix
        self.startTime = 0 #var
        self.machineID = ''  #var
        self.endTime = 0 #var
         
        for i in self.configure.index:
            setattr(self, i, self.configure[i])

    def set_machine_id(self, machineID):
        self.machineID = machineID

    def set_start_time(self, time):
        self.startTime = time
        processTime = int(self.processTime[ self.processTime["EQP_ID"] == self.machineID ]["PROCESS_TIME"])
        self.endTime = self.startTime + processTime
    
    def get_end_time(self):
        return self.endTime
    
    def __str__(self): 
        return json.dumps(self.configure.to_dict(), indent=4)
    
    def __repr__(self): 
        return json.dumps(self.configure.to_dict(), indent=4)


In [19]:
class Machine(object):
    def __init__(self, configure, setupTime):
        self.configure = configure
        self.setupTime = setupTime

        self.jobs = []

    def add_job(self, job):
        self.jobs.append(job)

    def sort_jobs(self):
        currentTime = int(self.configure["RECOVER_TIME"])
        print(currentTime)

        for i in range(len(self.jobs)):
            self.jobs[i].set_start_time(currentTime)
            currentTime = self.jobs[i].get_end_time()
            
machines = []
for i in range(len(tool.values)):
    machines.append(Machine(tool.iloc[i], setup_time.values))  #setuptime整張表

In [14]:
wip = pd.read_excel("./semiconductor_data.xlsx", sheet_name=2, dtype=str) #K

In [17]:
eqp = pd.read_excel("./semiconductor_data.xlsx", sheet_name=0, dtype=str) #K

In [7]:
setup_time = pd.read_excel("./semiconductor_data.xlsx", sheet_name=3, usecols="B:CW") #欄位
setup_time

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0,10,10,10,10,10,10,10,10,10,...,10,10,10,10,0,10,0,10,10,10
1,10,0,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
2,10,10,0,10,0,10,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
3,10,10,10,0,10,0,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
4,10,10,0,10,0,10,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,10,10,10,10,10,10,10,10,0,...,10,10,10,10,10,0,10,10,10,10
96,0,10,10,10,10,10,10,10,10,10,...,10,10,10,10,0,10,0,10,10,10
97,10,10,0,10,0,10,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
98,10,10,10,10,10,10,10,10,10,10,...,0,10,10,10,10,10,10,10,0,0


In [16]:
tool = pd.read_excel("./semiconductor_data.xlsx", sheet_name=1, dtype=str) #K

In [32]:
jobs = []                               #K
for i in range(len(wip.values)):
    jobs.append(Job(wip.iloc[i], eqp)) 
print(jobs[0].configure["LOT_ID"])

LOT001


In [11]:
machines = []
for i in range(len(tool.values)):
    machines.append(Machine(tool.iloc[i], setup_time.values))  #setuptime整張表

In [13]:
machines[1].sort_jobs()

240


In [80]:
RECIPE = pd.read_excel("./semiconductor_data.xlsx")

In [74]:
jobs[0].processTime

,EQP_ID,RECIPE,PROCESS_TIME
7,EQP001,Y8000,100
17,EQP002,Y8000,120
27,EQP003,Y8000,130
37,EQP004,Y8000,110
47,EQP005,Y8000,130
57,EQP006,Y8000,150
67,EQP007,Y8000,110
77,EQP008,Y8000,150
87,EQP009,Y8000,140
97,EQP010,Y8000,110


In [75]:
jobs[0].RECIPE

'Y8000'

In [6]:
a=[1,2,3]
len(a)

3

In [19]:
setup_time = pd.read_excel("./semiconductor_data.xlsx", sheet_name=3, usecols="B:CW") #欄位

In [5]:
setup_time.at[1,1]

10

In [37]:
setup_time = pd.read_excel("./semiconductor_data.xlsx", sheet_name=3) #欄位

In [44]:
import pandas as pd
result = pd.read_csv("result.csv")
result.columns=["Lot_id","HB","Wire","Entity","Start_t","End_t"]  # add header
entities= result.groupby("Entity") # group by entity
entities.groups #show dict, value is row index


{' BB211': [718, 719], ' BB212': [142, 253, 256, 346, 578], ' BB213': [691, 706, 707, 735], ' BB214': [8, 90, 107, 264, 301, 403, 434, 519, 623], ' BB215': [1919], ' BB216': [2063, 2087], ' BB217': [2143, 2147, 2150, 2160], ' BB218': [23, 47, 63, 89, 289, 447, 499, 506, 579], ' BB219': [2140, 2161], ' BB220': [1642, 1654], ' BB221': [2075, 2081, 2083], ' BB223': [58, 152, 184, 258, 262, 318, 430, 453, 480, 548, 581], ' BB224': [147, 195, 394, 542, 649], ' BB225': [24, 54, 215, 344, 391, 551, 595, 604, 642], ' BB226': [923, 924, 934, 977, 981, 991], ' BB227': [2146, 2151, 2156], ' BB228': [2155, 2158], ' BB229': [749], ' BB230': [2145, 2163], ' BB231': [49, 148, 313, 327, 362, 461, 625], ' BB232': [2061, 2062, 2074, 2076, 2077, 2079, 2080], ' BB233': [273, 395, 526, 568, 608], ' BB234': [1, 2, 182, 324, 452, 460, 486, 550, 582], ' BB235': [101, 306, 351, 400, 487, 594], ' BB441': [925, 975], ' BB442': [751], ' BB443': [1615, 1632, 1639], ' BB444': [1112, 1122, 1138, 1155], ' BB445': [11

In [104]:
import plotly.express as px
import pandas as pd
import datetime
import time # 引入time

result = pd.read_csv("result.csv",names=["Lot_id","HB","Wire","Entity","Start_t","End_t"])
entities= result.groupby("Entity") # group by entity
# entities.groups #show dict, value is row index
entity_key_list= list(entities.groups.keys()) # key list

# 2021/4/17 (改變數)
ini_time_stamp = 1618615800 

df=[]
for i in range(3):
    key= entity_key_list[i]
    for j in range(len(entities.groups[key])):
        row_index=entities.groups[key][j]
        Lot_id =result.iloc[row_index].at['Lot_id'].strip()
        Entity =result.iloc[row_index].at['Entity'].strip()
        start_ts =  time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ini_time_stamp+result.iloc[row_index].at['Start_t']*60)) 
        end_ts =  time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ini_time_stamp+result.iloc[row_index].at['End_t']*60)) 

        df.append(dict(Lot_ID= Lot_id, Start=start_ts,Finish=end_ts,Entity=Entity))

len(entity_key_list)
# #呈現圖表
# fig1 = px.timeline(df, x_start="Start", x_end="Finish", y="Entity", color="Wire",text="Lot_ID")
# fig1.update_yaxes(categoryorder="category descending") # 
# fig1.show()

543

In [119]:
import pandas as pd
df2 = pd.read_csv('WipOutPlanTime_.csv')
df3=df2[['wlot_lot_number','pin_package','wlot_prod','wlot_qty_1','bd_id']]
df2

,route,wlot_lot_number,wlot_crt_dat_al_1,pin_package,wlot_prod,wlot_qty_1,wlot_qty_2,wlot_oper,wlot_hold,cust_code,...,wlot_mvin_perfmd,wlot_actl_mvin_dt,wlot_actl_mvin_tm,wlot_dt_last_trans,wlot_tm_last_trans,wlot_hold_category,wlot_assy_lot,bd_id,urgent_code,wip_outplan_time
0,MBGA558,P10NVLK22,PHISON,uSD08AQV-C,0086ANV167,1551,0,2200,N,NV,...,Y,20210417,439,20210417,439,NaN,P10NVLK,ANV0086A1062A,E,2021/4/17 07:19
1,MBGA934,P13HQJR02,EMC,FPSSLGA28r6V,0282JHQ031,2360,0,2200,N,HQ,...,Y,20210417,644,20210417,644,NaN,P13HQJR,AHQ0282J2006A,NaN,2021/4/17 18:23
2,QFNS205,P15VGA437,ISSI-VG,DFN-08YD2A,008YDVG003,3520,0,2070,N,VG,...,Y,20210417,710,20210417,11708,NaN,P15VGA4,ABU008YD2008A,NaN,NaN
3,QFNS205,P15VGA453,ISSI-VG,DFN-08YD2A,008YDVG003,3520,0,2070,N,VG,...,Y,20210417,710,20210417,11708,NaN,P15VGA4,ABU008YD2008A,NaN,NaN
4,QFNS205,P15VGA421,ISSI-VG,DFN-08YD2A,008YDVG003,3520,0,2070,N,VG,...,Y,20210417,710,20210417,11708,NaN,P15VGA4,ABU008YD2008A,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32351,BGADAE01,ND9ABIX05,ChipMOS,MAT,0008WDAE001,4000,0,5070,N,FQ,...,N,0,0,20200928,230813,NaN,ND9ABIX,NaN,NaN,NaN
32352,BGADAE01,ND9ABIZ04,ChipMOS,MAT,0008WDAE001,4000,0,5070,N,FQ,...,N,0,0,20200930,25151,NaN,ND9ABIZ,NaN,NaN,NaN
32353,BGADAE01,ND9ASCR01,ChipMOS,MAT,0008WDAE001,11520,0,5070,N,FQ,...,N,0,0,20201010,23252,NaN,ND9ASCR,NaN,NaN,NaN
32354,BGADAE01,ND9ASCT05,ChipMOS,MAT,0008WDAE001,11520,0,5070,N,FQ,...,N,0,0,20201013,200440,NaN,ND9ASCT,NaN,NaN,NaN


In [416]:
import csv
# import pandas
import pandas as pd
import numpy as np  
# read csv data
df1 = pd.read_csv("result(new).csv",names=["wlot_lot_number","bd_id","HB","Wire","Entity","Start_t","End_t"])
df1.replace('\s+','',regex=True,inplace=True)
df2 = pd.read_csv('WipOutPlanTime_.csv')[['wlot_lot_number','wlot_crt_dat_al_1','pin_package','wlot_prod','wlot_qty_1']]
df3 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]
   
Left_join1 = pd.merge(df1, df2, on ='wlot_lot_number', how ='left')
Left_join2 = pd.merge(Left_join1, df3, on ='Entity', how ='left')
Left_join2
group= Left_join2.groupby(['wlot_crt_dat_al_1','pin_package','wlot_prod','bd_id','Location'])
group.describe()
#len(group.groups.keys())

#len(group.size())
list(dict(group.size()))
list(group.size())
list(dict(group.size()))
list(group.size())
group_keys_t=list(group.groups.keys())
#temp=list(list(group.groups.keys())[0])
#temp.append(5)
list(group_keys_t[0])

# all_rows=[]
# for i in range(len(group.size())):

#     temp= list(group_keys_t[i])
#     temp.append(group.size()[i]) #add amount ENT
#     temp.append(qty_list[i]) #add amount ENT
#     all_rows.append(temp)
# with open('yourOutput.csv', 'w', newline='') as csvFile:
#   # 建立 CSV 檔寫入器
#     writer = csv.writer(csvFile)
# #   # 1.直接寫出-標題
#     writer.writerow(['CUSTOMER','Pin-pkg','Prod_ID','Bond ID','TA/TB','Allocate ENT','Output plan'])

#     writer.writerows(all_rows)


In [461]:
import csv
import pandas as pd
import numpy as np  

# read csv data
df1 = pd.read_csv("result(new).csv",names=["wlot_lot_number","bd_id","HB","Wire","Entity","Start_t","End_t"])
df1.replace('\s+','',regex=True,inplace=True)

df2 = pd.read_csv('WipOutPlanTime_.csv')[['wlot_lot_number','wlot_crt_dat_al_1','pin_package','wlot_prod','wlot_qty_1']] ## replace to provided data

df3 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]
   
Left_join1 = pd.merge(df1, df2, on ='wlot_lot_number', how ='left')
Left_join2 = pd.merge(Left_join1, df3, on ='Entity', how ='left')
group= Left_join2.groupby(['wlot_crt_dat_al_1','pin_package','wlot_prod','bd_id','Location'])


# ---caculate total qty -------
qty_list=[]
group_key_list= list(group.groups.keys())

# accumulation
for i in range(len(group_key_list)): 
    key= group_key_list[i]
    qty_sum=0
    for j in range(len(group.groups[key])):
        row_index=group.groups[key][j]
        qty_sum+=Left_join1.iloc[row_index].at['wlot_qty_1']

    qty_list.append(qty_sum)
# -------

# Aggregate all needed data

# record original keys
group_keys_t=list(group.groups.keys())

all_rows=[]
for i in range(len(group.size())):

    temp= list(group_keys_t[i])
    temp.append(group.size()[i]) #add amount ENT column
    temp.append(qty_list[i]) #add amount qty column
    all_rows.append(temp)

# # write to csv
# with open('yourOutput.csv', 'w', newline='') as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(['CUSTOMER','Pin-pkg','Prod_ID','Bond ID','TA/TB','Allocate ENT','Output plan'])
#     writer.writerows(all_rows)



In [220]:
import csv

list1 =[[2,3],[6,7]]
# 開啟輸出的 CSV 檔案
with open('yourOutput.csv', 'w', newline='') as csvFile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvFile)

  # 1.直接寫出-標題
  writer.writerow(['姓名','年齡','電話','地址'])


  # 2.寫出多維陣列
  Table = [['姓名','年齡','電話','地址'],['葉大雄',18, '0911-123-123', '台北市火車站大廳'],['林靜香',26, '0911-456-456', '台北市中山北路二段']]
  writer.writerows(list1)


In [397]:
import csv
with open('yourOutput.csv', 'w', newline='') as csvFile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvFile)
  # 1.直接寫出-標題
  writer.writerow(['CUSTOMER','Pin-pkg','Prod_ID','Bond ID','TA/TB','Allocate ENT'])

  writer.writerows(list1)

In [17]:
## 0707
import csv
import pandas as pd
import numpy as np  

# read csv data
df1 = pd.read_csv("result(0707_2).csv")
df2 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]  
Left_join1 = pd.merge(df1, df2, left_on ='entity', right_on ='Entity', how='left')

group= Left_join1.groupby(['cust','pin_pkg','prod_id','bd_id','Location'])

# # # ---caculate total qty -------
# qty_list=[]
# group_key_list= list(group.groups.keys())

# # accumulation
# for i in range(len(group_key_list)): 
#     key= group_key_list[i]
#     qty_sum=0
#     for j in range(len(group.groups[key])):
#         row_index=group.groups[key][j]
#         qty_sum+=Left_join1.iloc[row_index].at['qty']

#     qty_list.append(qty_sum)
# # -------

# Aggregate all needed data

# record original keys
group_keys_t=list(group.groups.keys())

all_rows=[]
for i in range(len(group.size())):

    temp= list(group_keys_t[i])
    temp.append(group.size()[i]) #add amount ENT column
    #temp.append(qty_list[i]) #add amount qty column
    all_rows.append(temp)

# # write to csv
# with open('Output_0707.csv', 'w', newline='') as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(['CUSTOMER','Pin-pkg','Prod_ID','Bond ID','TA/TB','Allocate ENT','Output plan'])
#     writer.writerows(all_rows)

all_rows

[['A-DATA', 'STBGA252C4V', '252X2GJ006', 'AGJ252X20003A', 'TB-I', 4],
 ['A-DATA', 'STBGA252C4V', '252X2GJ006', 'AGJ252X20003A', 'TB-IP', 1],
 ['AKM', 'QFN-36AB1', '036ABBW009', 'ABW036AB1004A', 'TB-B', 4],
 ['AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-I', 8],
 ['AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-P', 12],
 ['AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-R', 9],
 ['AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-U', 45],
 ['CHIPLUS', 'MBGA48A2L', '048MAHD001', 'AHD048MA4002A', 'TA', 19],
 ['CYPRESS', 'TSOP2-54CMES', '054TSCS005', 'ACS054TS1012A', 'TA-2', 8],
 ['CYPRESS', 'TSOP2-54CMES', '054TSCS017', 'ACS054TS1009A', 'TA-2', 2],
 ['ChipMOS', 'DFN-08Y62D', '008Y6AW028', 'AAW008Y62014A', 'TB-P', 1],
 ['ChipMOS', 'DFN-08YD2A', '008YDAB014', 'AAB008YD2003A', 'TB-R', 3],
 ['ChipMOS', 'DFN-08YD2A', '008YDAB014', 'AAB008YD2003A', 'TB-U', 5],
 ['ChipMOS', 'DFN-08YD2A', '008YDVG006', 'ABU008YD2004A', 'TB-U', 1],
 ['ChipMOS',

In [3]:
import pandas as pd
df11 = pd.read_excel('product_find_process_id.xlsx')
df11

,product,pin_package,body_size,bd_id,route,process_id,bom_id,packing_id,gross_dies
0,0008WAB056,MAT,,ENG,MAT4,0008W343,8W000A01,ABX702A,2960
1,0008WAB099,MAT,,ENG,MAT4,162KE010,8W000A01,ABKE02A,2606
2,0008WAB100,MAT,(null),ENG,MAT444,0008W565,8W000A01,ABKE02A,2794
3,0008WAB178,MAT,,,MAT4,162KE022,8W000A01,ABKE02A,2606
4,0008WAW036,MAT,(null),ENG,MAT4,0008WS288,8W000A01,AWXY01A,1952
...,...,...,...,...,...,...,...,...,...
898,345XTNV038,STBGA345JAV,11.5*13*1.0,ANV345XT9001A,MBGA539,345XT024,XT345037,NVXT03A,124100
899,345XTNV062,STBGA345SBC,11.5*13*1.35,ANV345XTB007A,MBGA462,345XT043,XT345065,NVXT04A,124100
900,345XTNV063,STBGA345FBC,11.5*13*1.15,ANV345XTC003A,MBGA970,345XT044,XT345066,NVXT03A,3037
901,345XTNV066,STBGA345UBC,11.5*13*1.65,ANV345XTA010A,MBGA964,345XT046,XT345068,NVXT04A,3037


In [7]:
## 0707 test add time
import csv
import pandas as pd
import numpy as np  

## int time
relative_time_1 = 0        # >8:30
relative_time_2 = 150      # >11:00

# read csv data
df1 = pd.read_csv("result(0707_2).csv")
df2 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]  
result = pd.merge(df1, df2, left_on ='entity', right_on ='Entity', how='left')
group= result.groupby(['cust','pin_pkg','prod_id','bd_id','Location','entity'])


# # # ---caculate total qty -------
# qty_list=[]
# group_key_list= list(group.groups.keys())

# # accumulation
# for i in range(len(group_key_list)): 
#     key= group_key_list[i]
#     qty_sum=0
#     for j in range(len(group.groups[key])):
#         row_index=group.groups[key][j]
#         qty_sum+=result.iloc[row_index].at['qty']

#     qty_list.append(qty_sum)
# # -------


# # calculate each time of ENT


# # Aggregate all needed data


# # record original keys
# group_keys_t=list(group.groups.keys())

# all_rows=[]
# for i in range(len(group.size())):

#     temp= list(group_keys_t[i])
#     temp.append(group.size()[i]) #add amount ENT column
#     temp.append(qty_list[i]) #add amount qty column
#     all_rows.append(temp)

# # # write to csv
# # with open('Output_0707.csv', 'w', newline='') as csvFile:
# #     writer = csv.writer(csvFile)
# #     writer.writerow(['CUSTOMER','Pin-pkg','Prod_ID','Bond ID','TA/TB','Allocate ENT','Output plan'])
# #     writer.writerows(all_rows)
group.size()


cust    pin_pkg      prod_id     bd_id          Location  entity
A-DATA  STBGA252C4V  252X2GJ006  AGJ252X20003A  TB-I      BBA03     1
                                                          BBA15     1
                                                          BBA28     1
                                                          BBA31     1
                                                TB-IP     BBS06     1
                                                                   ..
ZJP     FBGA-96UEF   096EFYZ001  AYZ096EF2001A  TA        BB554     3
                                                          BB558     2
                                                          BB562     1
                                                          BB580     2
                                                          BB586     1
Length: 893, dtype: int64

In [22]:
## 0707 output entity 7/8 
import csv
import pandas as pd
import numpy as np  

## ini time
relative_time_1 = 0        # >8:30
relative_time_2 = 150      # >11:00

# read csv data
df1 = pd.read_csv("result(0707_2).csv")
df2 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]  
result = pd.merge(df1, df2, left_on ='entity', right_on ='Entity', how='left')
original_group=result.groupby(['cust','pin_pkg','prod_id','bd_id','Location'])

temp_group= result.groupby(['cust','pin_pkg','prod_id','bd_id','Location','entity']) 

# record key
temp_group_key_list= list(temp_group.groups.keys())
group_key_list= list(original_group.groups.keys())


# get all ENT (two times group))
temp_rows=[]
for i in range(len(temp_group.size())):
    temp0= list(list(temp_group.groups.keys())[i])
    temp_rows.append(temp0)
df_grp_entity = pd.DataFrame(temp_rows,columns =['cust','pin_pkg','prod_id','bd_id','Location','entity'])

grp_basis = df_grp_entity.groupby(['cust','pin_pkg','prod_id','bd_id','Location'])  

# # ---caculate total qty -------
qty_list=[]
# accumulation
for i in range(len(group_key_list)): 
    key= group_key_list[i]
    qty_sum=0
    for j in range(len(original_group.groups[key])):
        row_index=original_group.groups[key][j]
        qty_sum+=result.iloc[row_index].at['qty']
    qty_list.append(qty_sum)
# -------

# # calculate each time of ENT
result.iloc[2492].at['lot_number']
#grp_basis.groups[(group_key_list)]
temp_group.groups

#result.iloc[2491].at['lot_number']
# print(group_key_list[1])
# print(result.iloc[2492].at['lot_number'])
# # Aggregate all needed data
# all_rows=[]
# for i in range(len(original_group.size())):
#     temp= list(group_key_list[i])
#     temp.append(grp_basis.size()[i]) #add amount ENT column 4.1.1.2
#     temp.append(qty_list[i])

#     all_rows.append(temp)

# # write to csv
# with open('Output_0707_new.csv', 'w', newline='') as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(['CUSTOMER','Pin-pkg','Prod_ID','Bond ID','TA/TB','Allocate ENT','Output plan'])
#     writer.writerows(all_rows)
mid_term_marks_df.to_csv("midterm.csv", index=False)


{('A-DATA', 'STBGA252C4V', '252X2GJ006', 'AGJ252X20003A', 'TB-I', 'BBA03'): [2494], ('A-DATA', 'STBGA252C4V', '252X2GJ006', 'AGJ252X20003A', 'TB-I', 'BBA15'): [2495], ('A-DATA', 'STBGA252C4V', '252X2GJ006', 'AGJ252X20003A', 'TB-I', 'BBA28'): [2491], ('A-DATA', 'STBGA252C4V', '252X2GJ006', 'AGJ252X20003A', 'TB-I', 'BBA31'): [2493], ('A-DATA', 'STBGA252C4V', '252X2GJ006', 'AGJ252X20003A', 'TB-IP', 'BBS06'): [2492], ('AKM', 'QFN-36AB1', '036ABBW009', 'ABW036AB1004A', 'TB-B', 'BBF61'): [2314, 2315, 2316, 2318], ('AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-I', 'BBA19'): [1239, 1247, 1257, 1288], ('AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-I', 'BBA22'): [1264, 1281, 1284, 1285], ('AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-P', 'BBD03'): [1224, 1260, 1280], ('AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-P', 'BBD13'): [1231, 1242, 1252, 1291], ('AP Memory', 'DFN-08YM2G', '008YMQW005', 'AQW008YM2003A', 'TB-P', 'BBD14'): [125

In [5]:
## 0707 test add time
import csv
import pandas as pd
import numpy as np  



# read csv data
df1 = pd.read_csv("Output_0707_entity.csv")
group= df1.groupby(['CUSTOMER','Pin-pkg','Prod_ID','Bond ID','TA/TB'])
sum=0
for i in range(len(group.size())):
    sum+=group.size()[i]
group.size()

CUSTOMER   Pin-pkg      Prod_ID     Bond ID        TA/TB
A-DATA     STBGA252C4V  252X2GJ006  AGJ252X20003A  TB-I      4
                                                   TB-IP     1
AKM        QFN-36AB1    036ABBW009  ABW036AB1004A  TB-B      1
AP Memory  DFN-08YM2G   008YMQW005  AQW008YM2003A  TB-I      2
                                                   TB-P      4
                                                            ..
WINBOND    VFBGA60B4L   060NRAB001  AB060NR1001A   TA-3      4
                        060NRAB005  AAB060NR1004B  TA        1
                                                   TA-2      1
           VFBGA90B4L   090NCAB019  AAB090NC4006A  TA        1
ZJP        FBGA-96UEF   096EFYZ001  AYZ096EF2001A  TA       11
Length: 157, dtype: int64

In [24]:
import pandas as pd
import numpy as np  
df1 = pd.DataFrame({'set':['A','B','C','D'],
                        'ent_all':[3,2,7,5]})
df2 = pd.DataFrame({'set':['A','C','D'],
                        'ent_11':[2,1,1]})                       
result = pd.merge(df1, df2, on ='set', how ='left')
result
g= result.fillna(0)  #填空值
g

,set,ent_all,ent_11
0,A,3,2.0
1,B,2,0.0
2,C,7,1.0
3,D,5,1.0


In [25]:
## 7/9
import csv
import pandas as pd
import numpy as np  
from datetime import datetime

## ini time
relative_time_1 = 0      # >08:30 改07:30
relative_time_2 = 150      # >11:00

# read csv data
df1 = pd.read_csv("result.csv")
df2 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]  
result = pd.merge(df1, df2, left_on ='entity', right_on ='Entity', how='left')
original_group=result.groupby(['cust','pin_pkg','prod_id','bd_id','Location']) 

# # 11. 7:30
def get_result_byFilter(result,relative_time):
    filter_strt = result["start_time"] <= relative_time #0, 150
    filter_end = result["end_time"] >= relative_time
    filter_result = result[filter_strt & filter_end]

    return filter_result

# 
def getENTdf(temp_result,ent_colname):

    temp_group= temp_result.groupby(['cust','pin_pkg','prod_id','bd_id','Location','entity']) 
    df_grp_entity = pd.DataFrame(temp_group.groups.keys())
    grp_basis = df_grp_entity.groupby([0,1,2,3,4])  # cust > Location
    df_keys= pd.DataFrame(grp_basis.groups.keys())
    df_Ent_amount= pd.DataFrame(list(grp_basis.size()))
    ENTdf= pd.concat([df_keys, df_Ent_amount], axis=1)
    ENTdf.columns=['cust','pin_pkg','prod_id','bd_id','Location',ent_colname]

    return ENTdf

ENT_all = getENTdf(result,'Allocate ENT')
ENT_08 = getENTdf(get_result_byFilter(result,relative_time_1),'Original ENT(08:30)') # 改
ENT_11 = getENTdf(get_result_byFilter(result,relative_time_2),'Original ENT(11:00)')

# # ---caculate total qty -------
qty_list=[]
group_key_list= list(original_group.groups.keys())

# accumulation
for i in range(len(group_key_list)): 
    key= group_key_list[i]
    qty_sum=0
    for j in range(len(original_group.groups[key])):
        row_index=original_group.groups[key][j]
        qty_sum+=result.iloc[row_index].at['qty']

    qty_list.append(qty_sum)
df_qty = pd.DataFrame(qty_list,columns=['Output plan'])
# -------

# merge three df
result_all_8 = pd.merge(ENT_all, ENT_08, on =(['cust','pin_pkg','prod_id','bd_id','Location']), how ='left')
ENT_result = pd.merge(result_all_8, ENT_11, on =(['cust','pin_pkg','prod_id','bd_id','Location']), how ='left')
# concat all
temp_df= pd.concat([ENT_result, df_qty], axis=1)

## change column sequence
final_df = temp_df[['cust','pin_pkg','prod_id','bd_id','Location','Original ENT(08:30)','Original ENT(11:00)','Allocate ENT','Output plan']]

# # write to csv
final_df.to_csv(foldername +"simulation_output.csv", index=False ,na_rep=0) 

In [30]:
# output2 

from datetime import datetime
import time # 
import pandas as pd
df1 = pd.read_csv("result.csv")
# 2021/4/17 08:30 (改 變數)
ini_time_stamp = 1618619400 

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory) 

foldername = './data_result' + datetime.now().strftime('-%Y%m%d%H%M') + '/' # folder which store result picture
createFolder(foldername)  

df_output2 = df1.copy(deep=True)
for i in range(len(df2.index)):
    df_output2['start_time'].iloc[i] =  time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ini_time_stamp+df1['start_time'].iloc[i]*60))
    df_output2['end_time'].iloc[i] =  time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ini_time_stamp+df1['end_time'].iloc[i]*60)) 

df_output2= df_output2[['lot_number','cust','pin_pkg','prod_id','qty','part_id','part_no','bd_id','entity','start_time','end_time']]
df_output2.to_csv(foldername +"lots_output.csv", index=False ,na_rep=0)



C:\Users\LiangCJ\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
## test use index to groupby
import csv
import pandas as pd
import numpy as np  

## ini time
relative_time_1 = 0        # >8:30
relative_time_2 = 150      # >11:00

# read csv data
df1 = pd.read_csv("result(0707_2).csv")
df2 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]  
result = pd.merge(df1, df2, left_on ='entity', right_on ='Entity', how='left')
original_group=result.groupby(['cust','pin_pkg','prod_id','bd_id','Location'])

temp_group= result.groupby(['cust','pin_pkg','prod_id','bd_id','Location','entity'])

df_grp_entity = pd.DataFrame(temp_group.groups.keys())
grp_basis = df_grp_entity.groupby([0,1])  
grp_basis.size()


0          1           
A-DATA     STBGA252C4V      5
AKM        QFN-36AB1        1
AP Memory  DFN-08YM2G      25
CHIPLUS    MBGA48A2L        5
CYPRESS    TSOP2-54CMES     3
                           ..
WINBOND    TSOP2-54ME      21
           TSOP2-66ME       3
           VFBGA60B4L      10
           VFBGA90B4L       1
ZJP        FBGA-96UEF      11
Length: 67, dtype: int64

In [ ]:
## 0707 output entity 7/8 
import csv
import pandas as pd
import numpy as np  

## ini time
relative_time_1 = 0        # >8:30
relative_time_2 = 150      # >11:00

# read csv data
df1 = pd.read_csv("result(0707_2).csv")
df2 = pd.read_excel('機台區域作業產品.xls')[['Entity','Location']]  
result = pd.merge(df1, df2, left_on ='entity', right_on ='Entity', how='left')
original_group=result.groupby(['cust','pin_pkg','prod_id','bd_id','Location'])

# 
temp_group= result.groupby(['cust','pin_pkg','prod_id','bd_id','Location','entity']) 
df_grp_entity = pd.DataFrame(temp_group.groups.keys(),columns =['cust','pin_pkg','prod_id','bd_id','Location','entity'])
grp_basis = df_grp_entity.groupby(['cust','pin_pkg','prod_id','bd_id','Location'])  

# 11.
filter1 = result["start_time"] <= 150
filter2 = result["end_time"] >= 150
result2 = result[filter1 & filter2]


temp_group2 = result2.groupby(['cust','pin_pkg','prod_id','bd_id','Location','entity']) 
df_grp_entity2 = pd.DataFrame(temp_group2.groups.keys(),columns =['cust','pin_pkg','prod_id','bd_id','Location','entity'])
grp_basis2 = df_grp_entity2.groupby(['cust','pin_pkg','prod_id','bd_id','Location'])


dd1= pd.DataFrame(grp_basis.groups.keys())
ddd1= pd.DataFrame(list(grp_basis.size()))
a0 = pd.concat([dd1, ddd1], axis=1)
a0.columns=['cust','pin_pkg','prod_id','bd_id','Location','Allocate ENT']

dd= pd.DataFrame(grp_basis2.groups.keys())
ddd= pd.DataFrame(list(grp_basis2.size()))
a = pd.concat([dd, ddd], axis=1)
a.columns=['cust','pin_pkg','prod_id','bd_id','Location','Original ENT(11:00)']

result__ = pd.merge(a0, a, on =(['cust','pin_pkg','prod_id','bd_id','Location']), how ='left')
# 待驗證
# change column sequence
new = result__[['cust','pin_pkg','prod_id','bd_id','Location','Original ENT(11:00)','Allocate ENT']]
new1 = new.fillna(0)
new1

In [21]:
## Gantt


import plotly.express as px
import pandas as pd
import datetime

df=[]
color_discrete_sequence_list=['#F0F8FF','#FAEBD7','#00FFFF','#7FFFD4','#F0FFFF','#F5F5DC','#FFE4C4',
            '#000000','#FFEBCD','#0000FF','#8A2BE2','#A52A2A','#DEB887','#5F9EA0',
            '#7FFF00','#D2691E','#FF7F50','#6495ED','#FFF8DC','#DC143C','#00FFFF',
            '#00008B','#008B8B','#B8860B','#A9A9A9','#006400','#BDB76B','#8B008B',
            '#556B2F','#FF8C00','#9932CC','#8B0000','#E9967A','#8FBC8F','#483D8B',
            '#2F4F4F','#00CED1','#9400D3','#FF1493','#00BFFF','#696969','#1E90FF',
            '#B22222','#FFFAF0','#228B22','#FF00FF','#DCDCDC','#F8F8FF','#FFD700',
            '#DAA520','#808080','#008000','#ADFF2F','#F0FFF0','#FF69B4','#CD5C5C',
            '#4B0082','#FFFFF0','#F0E68C','#E6E6FA','#FFF0F5','#7CFC00','#FFFACD',
            '#ADD8E6','#F08080','#E0FFFF','#FAFAD2','#90EE90','#D3D3D3','#FFB6C1',
            '#FFA07A','#20B2AA','#87CEFA','#778899','#B0C4DE','#FFFFE0','#00FF00',
            '#32CD32','#FAF0E6','#FF00FF','#800000','#66CDAA','#0000CD','#BA55D3',
            '#9370DB','#3CB371','#7B68EE','#00FA9A','#48D1CC','#C71585','#191970',
            '#F5FFFA','#FFE4E1','#FFE4B5','#FFDEAD','#000080','#FDF5E6','#808000',
            '#6B8E23','#FFA500','#FF4500','#DA70D6','#EEE8AA','#98FB98','#AFEEEE',
            '#DB7093','#FFEFD5','#FFDAB9','#CD853F','#FFC0CB','#DDA0DD','#B0E0E6',
            '#800080','#FF0000','#BC8F8F','#4169E1','#8B4513','#FA8072','#FAA460',
            '#2E8B57','#FFF5EE','#A0522D','#C0C0C0','#87CEEB','#6A5ACD','#708090',
            '#FFFAFA','#00FF7F','#4682B4','#D2B48C','#008080','#D8BFD8','#FF6347',
            '#40E0D0','#EE82EE','#F5DEB3','#FFFFFF','#F5F5F5','#FFFF00','#9ACD32']

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job ", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
])
Map={'Job A':'#F0F8FF','Job B':'#FAEBD7','Job C':'#9932CC'}

#呈現圖表
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Task",color_discrete_sequence=color_discrete_sequence_list ,text="Task")
fig.show()

In [19]:
## 0707 output entity 7/8 
import csv
import pandas as pd
import numpy as np  

## ini time
relative_time_1 = 0        # >8:30
relative_time_2 = 150      # >11:00

# read csv data
df1 = pd.read_csv("result(0707_2).csv")
result=df1.groupby('bd_id')
result.size()
px.colors.qualitative.Plotly 

['#636EFA',
 '#EF553B',
 '#00CC96',
 '#AB63FA',
 '#FFA15A',
 '#19D3F3',
 '#FF6692',
 '#B6E880',
 '#FF97FF',
 '#FECB52',
 '#000000']